In [ ]:
from pathlib import Path

#%matplotlib inline
%matplotlib ipympl
import matplotlib.pyplot as plt
import numpy as np

import sed
from sed import SedProcessor

In [ ]:
local_path = Path(sed.__file__).parent.parent / "tutorial/"
config_file = local_path / "sxp_config.yaml"
assert config_file.exists()

# Loading Data

In [ ]:
config = {
    "core": {
        "paths": {
            "data_raw_dir": "../..//GPFS/exfel/exp/SXP/202302/p004316/raw/",
            "data_parquet_dir": "../../sxp_parquet/",
        }
    }
}
sp = SedProcessor(
    runs=["0058", "0059", "0060", "0061"],
    config=config,
    user_config=config_file,
    system_config={},
    collect_metadata=False,
    #force_recreate=True,
)

In [ ]:
plt.figure()
ids=sp.dataframe.trainId.compute().values
plt.plot(ids)
plt.show()

In [ ]:
sp.dataframe.head()

In [ ]:
sp.filter_column("pulseId", max_value=300)

In [ ]:
axes = ["dldPosX", "dldPosY", "dldTimeSteps", "delayStage"]
bins = [100, 100, 100, 100]
ranges = [(0, 4000), (0, 4000), (1000, 28000), (-1000, 1000)]
sp.view_event_histogram(dfpid=3, axes=axes, bins=bins, ranges=ranges)

In [ ]:
axes = ["pulseId", "electronId"]
bins = [51, 11]
ranges = [(-0.5, 50.5), (-0.5, 10.5)]
sp.view_event_histogram(dfpid=1, axes=axes, bins=bins, ranges=ranges)

In [ ]:
axes = ["dldTimeSteps", "pulseId"]
bins = [200, 800]
ranges = [(8000, 28000), (0, 800)]
res3 = sp.compute(bins=bins, axes=axes, ranges=ranges)
plt.figure()
res3.plot(robust=True)

In [ ]:
sp.add_jitter()

# time-of-flight spectrum

In [ ]:
sp.filter_column("electronId", max_value=1)

In [ ]:
axes = ["dldPosX", "dldPosY", "dldTimeSteps", "delayStage"]
bins = [100, 100, 100, 100]
ranges = [(0, 4000), (0, 4000), (8000, 28000), (-200, 200)]
res = sp.compute(bins=bins, axes=axes, ranges=ranges, normalize_to_acquisition_time="delayStage")

In [ ]:
plt.figure()
res.loc[{"dldPosX":slice(100, 3600), "dldPosY":slice(100,3500)}].sum(axis=(0,1)).plot()

In [ ]:
plt.figure()
plt.plot(res.dldTimeSteps*0.00685, res.sum(axis=(0,1)))

In [ ]:
plt.figure()
res.loc[{'dldTimeSteps':slice(8000, 20000)}].sum(axis=2).plot()

# Energy Calibration

In [ ]:
sp.append_tof_ns_axis()

In [ ]:
axes = ['dldTime']
bins = [2000]
ranges = [[10,200]]
res = sp.compute(bins=bins, axes=axes, ranges=ranges)
plt.figure()
res.plot()

In [ ]:
import xarray as xr
runs = ["0074", "0073", "0072", "0071", "0070", "0064", "0065", "0066", "0067", "0068", "0069"]
biases = np.arange(962, 951, -1)
data = []
for run in runs:
    sp.load(runs=[run])
    axes = ["dldTimeSteps"]
    bins = [2000]
    ranges = [(1000, 25000)]
    res = sp.compute(bins=bins, axes=axes, ranges=ranges)
    data.append(res)
    res.plot()

biasSeries = xr.concat(data, dim=xr.DataArray(biases, dims="sampleBias", name="sampleBias"))

In [ ]:
sp.load_bias_series(binned_data=biasSeries)

In [ ]:
ranges=(6380, 6700)
ref_id=6
sp.find_bias_peaks(ranges=ranges, ref_id=ref_id, apply=True)

In [ ]:
sp.calibrate_energy_axis(
    ref_id=5,
    ref_energy=-33,
    method="lmfit",
    energy_scale='kinetic',
    d={'value':1.1,'min': .2, 'max':5.0, 'vary':False},
    t0={'value':-1E-8, 'min': -1E-6, 'max': 1e-4, 'vary':True},
    E0={'value': 0., 'min': -1500, 'max': 1500, 'vary': True},
)

In [ ]:
sp.save_energy_calibration()

In [ ]:
sp.append_energy_axis()

In [ ]:
sp.load(runs=["0058", "0059", "0060", "0061"])
sp.add_jitter()
sp.append_energy_axis()

In [ ]:
axes = ['energy', "delayStage"]
bins = [200, 100]
ranges = [[-37,-31], [-135, -115]]
res = sp.compute(bins=bins, axes=axes, ranges=ranges, normalize_to_acquisition_time="delayStage")

In [ ]:
plt.figure()
res.plot()
plt.title("Scans 58-61 binned to energy and delayStage")